In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob
import librosa
import librosa.display
from sklearn.linear_model import LogisticRegression
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
import glob
import soundfile as sf
import pickle
import sys

c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '


In [2]:
tqdm.pandas(desc="!")
np.set_printoptions(threshold=sys.maxsize)

c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [39]:
train_classified = pd.read_pickle("train_classified")
train_classified["length"] = 0

In [40]:
train_classified

,sample_rate,species_id,waveform,length
0,48000.0,14.0,"[0.001678466796875, 0.0067138671875, -0.002929...",0
1,48000.0,23.0,"[0.021209716796875, 0.018310546875, 0.01879882...",0
2,48000.0,12.0,"[-0.011444091796875, -0.000396728515625, 0.015...",0
3,48000.0,17.0,"[0.0008544921875, 0.001953125, 0.0043334960937...",0
4,48000.0,10.0,"[0.0009765625, 0.002899169921875, 0.0007934570...",0
...,...,...,...,...
1211,48000.0,13.0,"[0.015350341796875, 0.032379150390625, 0.02734...",0
1212,48000.0,4.0,"[-0.03717041015625, -0.024932861328125, -0.008...",0
1213,48000.0,0.0,"[0.019012451171875, 0.015106201171875, 0.00924...",0
1214,48000.0,5.0,"[-0.014678955078125, -0.009765625, -0.01284790...",0


In [41]:

for i in range(len(train_classified)):
    train_classified.iloc[i, train_classified.columns.get_loc("length")] = len(train_classified.iloc[i]["waveform"])

In [42]:
train_classified.species_id.unique().astype(int).astype(str)

array(['14', '23', '12', '17', '10', '8', '0', '18', '15', '1', '6', '7',
       '11', '13', '16', '19', '3', '2', '4', '22', '9', '21', '20', '5'],
      dtype='<U11')

In [43]:

len(train_classified.iloc[2]['waveform'])
train_classified.head()

,sample_rate,species_id,waveform,length
0,48000.0,14.0,"[0.001678466796875, 0.0067138671875, -0.002929...",28161
1,48000.0,23.0,"[0.021209716796875, 0.018310546875, 0.01879882...",292017
2,48000.0,12.0,"[-0.011444091796875, -0.000396728515625, 0.015...",150529
3,48000.0,17.0,"[0.0008544921875, 0.001953125, 0.0043334960937...",181392
4,48000.0,10.0,"[0.0009765625, 0.002899169921875, 0.0007934570...",117307


In [44]:
train_classified['length'].mean()

121781.69243421052

In [13]:
# cut off at 44.1k sample rate /. 2. 
# mapping from time domain into frequency domain
# https://www.nti-audio.com/en/support/know-how/fast-fourier-transform-fft

# y = f(x)

def extract_features( row ):
    samplerate = 48000
    data = row['waveform']
    varfft = np.abs( np.fft.fft(a=data)[:(len(data)//2)] )
    x = np.linspace(0, len(varfft), num=len(varfft), endpoint=True)
    f1 = interp1d(x, varfft, kind='cubic')
    x = np.linspace(0, len(varfft), num=1000, endpoint=True)
    varfft = f1(x)
    
    return varfft

In [46]:
train_classified['fft_1d'] = train_classified.progress_apply(extract_features, axis=1)

In [15]:
TRAIN = []
TARGET = []
for i in range(len(train_classified)):
    entry = train_classified["fft_1d"].iloc[i]
    t = train_classified["species_id"].iloc[i]
    TRAIN.append(entry)
    TARGET.append(t)

In [11]:
# Basic Log Reg
X = train_classified.loc[:, train_classified.columns == "fft_1d"]
y = train_classified.loc[:, train_classified.columns == "species_id"]

In [12]:

X_train, X_test, y_train, y_test = train_test_split(TRAIN, TARGET)


In [13]:
logreg = LogisticRegression( max_iter=5000 )
logreg.fit(TRAIN, TARGET)

LogisticRegression(max_iter=5000)

In [43]:
logreg.score(X_test, y_test)

1.0

In [41]:
logreg_pickle = open("logreg.pkl", 'wb')
pickle.dump(logreg, logreg_pickle)
# Close the pickle instances
logreg_pickle.close()

In [26]:
logreg_pickle = open("logreg.pkl", 'rb')
logreg = pickle.load(logreg_pickle)

In [3]:
testfiles = glob.glob( 'data/test/*.flac' )

In [8]:
testfiles[0]

'data/test\\000316da7.flac'

In [17]:
testfiles[1][10:19]

'003bc2cb2'

In [49]:
train_classified.to_hdf(r'train_classified_logreg.h5', key='stage', mode='w')

c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['waveform', 'fft_1d'], dtype='object')]

  encoding=encoding,


In [6]:
train_classified = pd.read_hdf("train_classified_logreg.h5", key='stage')

In [7]:
train_classified

,sample_rate,species_id,waveform,length,fft_1d
0,48000.0,14.0,"[0.001678466796875, 0.0067138671875, -0.002929...",28161,"[1.55487060546875, 2.152843965490996, 1.108953..."
1,48000.0,23.0,"[0.021209716796875, 0.018310546875, 0.01879882...",292017,"[3.5285034179687558, 65.4764965268222, 39.9461..."
2,48000.0,12.0,"[-0.011444091796875, -0.000396728515625, 0.015...",150529,"[3.7118530273437456, 21.142677414976653, 37.19..."
3,48000.0,17.0,"[0.0008544921875, 0.001953125, 0.0043334960937...",181392,"[0.3219299316406248, 4.884913960046954, 9.0780..."
4,48000.0,10.0,"[0.0009765625, 0.002899169921875, 0.0007934570...",117307,"[0.9687805175781222, 9.730580973532323, 8.2289..."
...,...,...,...,...,...
1211,48000.0,13.0,"[0.015350341796875, 0.032379150390625, 0.02734...",29952,"[0.5283203125, 1.256245900253151, 1.4916762053..."
1212,48000.0,4.0,"[-0.03717041015625, -0.024932861328125, -0.008...",104957,"[4.819915771484377, 3.0524111841585384, 2.7690..."
1213,48000.0,0.0,"[0.019012451171875, 0.015106201171875, 0.00924...",38141,"[2.2714538574218763, 2.480650746062264, 5.2153..."
1214,48000.0,5.0,"[-0.014678955078125, -0.009765625, -0.01284790...",112896,"[0.880950927734375, 38.33358261644435, 16.2084..."


In [4]:
test_df = pd.DataFrame()

In [5]:
for i in tqdm(range(len(testfiles))):
    waveform, sample_rate = sf.read(testfiles[i])
    test_df=test_df.append({"waveform" : waveform, "recording_id" : testfiles[i][10:19]}, ignore_index=True)
    

In [14]:
testfiles = None

In [25]:
test_df['length'] = test_df.apply( lambda row: len(row['waveform']), axis=1)

In [6]:
test_df.to_hdf("testfiles_df.hd5", key='stage', mode='w')

c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['recording_id', 'waveform'], dtype='object')]

  encoding=encoding,


MemoryError: 

In [23]:
test_df = pd.read_hdf("testfiles_df.hd5", key='stage')

In [7]:
test_df

,recording_id,waveform
0,000316da7,"[-0.002532958984375, 0.00115966796875, -0.0045..."
1,003bc2cb2,"[0.062347412109375, -0.02972412109375, 0.00027..."
2,0061c037e,"[-0.005126953125, -0.004608154296875, -0.00811..."
3,010eb14d3,"[-0.02447509765625, 0.011688232421875, 0.03973..."
4,011318064,"[0.00128173828125, -0.002227783203125, 0.00207..."
...,...,...
1987,ff68f3ac3,"[0.014617919921875, 0.0115966796875, -0.003601..."
1988,ff973e852,"[0.0135498046875, -0.003509521484375, -0.00485..."
1989,ffa5cf6d6,"[0.001373291015625, -0.003265380859375, 0.0014..."
1990,ffa88cbb8,"[-0.005706787109375, 0.003143310546875, 3.0517..."


In [8]:

test_slice_df = pd.DataFrame()

In [9]:
# x + 121781
# x = [0 -> length- 121781]


for i in tqdm(range(len(test_df))):
    entry = test_df.iloc[i]
    waveform = entry["waveform"]
    x = 0
    c_length = 0
    while len(entry['waveform']) - x - c_length > 0:
#         print("true")
        c_length = (len(entry['waveform']) - x) if (len(entry['waveform']) - x) < 121781 else 121781
        test_slice_df = test_slice_df.append({"waveform": waveform[x : x+c_length], "recording_id" : entry['recording_id']}, ignore_index=True)
        
        x = x+ c_length

In [10]:
test_slice_df.to_pickle("sliced_testfiles.pkl")

MemoryError: 

In [11]:
type(test_df.iloc[0].waveform)

numpy.ndarray

In [15]:
def my_reset(*varnames):
    """
    varnames are what you want to keep
    """
    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

In [25]:
my_reset(TEST, ids)

TypeError: unhashable type: 'list'

In [14]:
test_slice_df['fft_1d'] = test_slice_df.progress_apply(extract_features, axis=1)

In [21]:
test_slice_df.to_hdf("D:/test_slice.hd5", key='stage', mode='w')

MemoryError: 

In [22]:
TEST = []
ids = []
for i in range(len(test_slice_df)):
    entry = test_slice_df["fft_1d"].iloc[i]
    entry2 = test_slice_df["recording_id"].iloc[i]
    ids.append(entry2)
    TEST.append(entry)


In [23]:
len(ids)

45816

In [27]:
Y_pp = pd.DataFrame(logreg.predict_proba(TEST), 
             columns=['s14', 's23', 's12', 's17', 's10', 's8', 's0', 's18', 's15', 's1', 's6', 's7',
       's11', 's13', 's16', 's19', 's3', 's2', 's4', 's22', 's9', 's21', 's20', 's5'])
Y_pp['recording_id'] = ids
Y_pp.tail()

,s14,s23,s12,s17,s10,s8,s0,s18,s15,s1,...,s19,s3,s2,s4,s22,s9,s21,s20,s5,recording_id
45811,0.000150,0.002069,0.539520,0.000384,0.003178,0.110665,0.000450,0.030998,0.000948,0.015237,...,0.001841,0.014402,0.003899,0.000086,0.003334,0.001575,0.000124,0.080442,0.001357,ffda5d7b3
45812,0.000726,0.000512,0.059974,0.000028,0.002408,0.002370,0.002753,0.000426,0.000110,0.002329,...,0.000474,0.006718,0.020542,0.000012,0.017479,0.000247,0.000129,0.486626,0.000067,ffda5d7b3
45813,0.000113,0.004178,0.021621,0.000015,0.012075,0.005216,0.000778,0.002895,0.000097,0.063635,...,0.047720,0.013224,0.125311,0.000069,0.016064,0.001106,0.000564,0.292765,0.000071,ffda5d7b3
45814,0.000058,0.001644,0.005724,0.000108,0.004375,0.000801,0.003874,0.008024,0.000079,0.002926,...,0.001363,0.024770,0.022506,0.000047,0.379647,0.007637,0.000113,0.015567,0.000092,ffda5d7b3
45815,0.000004,0.000043,0.000370,0.000003,0.001326,0.000065,0.000058,0.000189,0.000021,0.001461,...,0.002166,0.046410,0.006107,0.000002,0.001179,0.000151,0.000015,0.000182,0.000008,ffda5d7b3


In [28]:
Y_pp.to_hdf("probs.hd5", key ="stage", mode="w")

In [29]:
Y_pp

,s14,s23,s12,s17,s10,s8,s0,s18,s15,s1,...,s19,s3,s2,s4,s22,s9,s21,s20,s5,recording_id
0,2.696585e-06,9.606434e-06,2.099302e-01,7.508733e-10,0.000154,3.994243e-01,0.016123,2.640657e-03,3.733729e-05,3.473008e-01,...,3.373990e-03,2.960197e-05,4.010835e-03,2.223056e-07,4.181526e-04,1.345791e-04,1.988589e-06,9.590037e-05,3.227087e-05,000316da7
1,4.912548e-05,6.421613e-08,3.408230e-04,4.946905e-09,0.085651,6.145341e-02,0.675009,5.904013e-05,1.869274e-03,7.423019e-03,...,1.309623e-02,1.152142e-08,3.575960e-05,2.502622e-06,1.198872e-09,1.165217e-12,7.321308e-03,6.388191e-07,1.762114e-06,000316da7
2,3.100416e-06,9.000835e-06,1.475923e-01,1.861877e-06,0.001495,2.593281e-01,0.136891,1.300409e-01,1.752284e-04,2.698334e-01,...,3.045747e-02,8.623361e-04,1.545470e-03,8.103187e-08,6.969422e-04,1.113279e-03,2.353779e-04,1.463367e-03,2.363503e-05,000316da7
3,2.135434e-08,1.375040e-14,2.220291e-12,2.570992e-15,0.000214,3.839769e-09,0.999751,1.552093e-09,3.644253e-07,3.224120e-07,...,4.864768e-09,7.906672e-15,9.102649e-15,1.029907e-11,3.112845e-16,2.520704e-18,4.402937e-06,1.541666e-13,2.371294e-09,000316da7
4,4.725426e-06,7.935605e-09,9.525863e-04,6.500104e-15,0.000038,1.439879e-03,0.996567,3.319492e-06,1.456934e-04,4.874330e-04,...,1.826172e-07,6.343393e-09,4.469797e-05,3.271188e-06,1.082072e-05,1.778421e-12,3.923372e-08,4.391162e-08,1.780008e-07,000316da7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45811,1.501330e-04,2.068931e-03,5.395197e-01,3.840627e-04,0.003178,1.106653e-01,0.000450,3.099753e-02,9.476007e-04,1.523682e-02,...,1.840526e-03,1.440158e-02,3.899313e-03,8.639609e-05,3.334151e-03,1.575393e-03,1.244992e-04,8.044198e-02,1.356940e-03,ffda5d7b3
45812,7.262438e-04,5.124223e-04,5.997392e-02,2.771871e-05,0.002408,2.370453e-03,0.002753,4.264678e-04,1.099829e-04,2.328544e-03,...,4.744631e-04,6.717636e-03,2.054184e-02,1.183047e-05,1.747929e-02,2.470205e-04,1.291725e-04,4.866264e-01,6.654796e-05,ffda5d7b3
45813,1.128003e-04,4.177626e-03,2.162111e-02,1.538027e-05,0.012075,5.216006e-03,0.000778,2.895419e-03,9.665929e-05,6.363519e-02,...,4.772010e-02,1.322351e-02,1.253112e-01,6.873276e-05,1.606430e-02,1.105656e-03,5.644207e-04,2.927647e-01,7.107662e-05,ffda5d7b3
45814,5.821679e-05,1.643948e-03,5.723688e-03,1.076741e-04,0.004375,8.009150e-04,0.003874,8.024393e-03,7.901919e-05,2.925859e-03,...,1.363212e-03,2.477005e-02,2.250575e-02,4.663243e-05,3.796471e-01,7.637086e-03,1.127604e-04,1.556741e-02,9.156995e-05,ffda5d7b3


In [30]:
softmax = Y_pp.idxmax(axis=1)

TypeError: reduction operation 'argmax' not allowed for this dtype

In [32]:
softmax

0        s8
1        s8
2        s8
3       s16
4        s8
       ... 
7631     s8
7632     s8
7633     s8
7634    s16
7635     s8
Length: 7636, dtype: object